# Robot search

In this notebook we introduce the RL framework for target searching with robots. It is based on the agents and environments presented in `rl_opts.rl_framework`. Some features of the code are:

- we use `numba` to improve speed.
- we implement more efficient ways of updating the H and G matrix (contribution by Dr. Michele Caraglio).
- we consider as base case `num_agents = 1`. 
- we modify the old version of our agents to make them more realistic and more similar to the dynamics of real robots.

In [ ]:
#| default_exp robot_search

#| hide
# Libs

Input libraries

In [ ]:
#| export
import numpy as np
import numba
from numba.experimental import jitclass
from numba import jit, float64, int64, bool_, prange, njit
import math
import random
#from rl_opts.utils import isBetween_c_Vec, coord_mod
NOPYTHON = True

In [ ]:
#| hide
#| exec: false
# for debugging
NOPYTHON = False

In [ ]:
# | hide
import matplotlib.pyplot as plt
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Helpers

## Crossing targets in straight line (isBetween)

Function to evaluate whether the robot walking in a straight line from point a to point b crosses point c (targets).

In [ ]:
# |export
@jit(nopython = NOPYTHON)
def isBetween_c_vec_numba(a, b, c, r):
        """
        Checks whether point c is crossing the line formed with point a and b.

        Parameters
        ----------
        a : tensor, shape = (1,2)
            Previous position.
        b : tensor, shape = (1,2)
            Current position.
        c : tensor, shape = (Nt,2)
            Positions of all targets.
        r : int/float
            Target radius.

        Returns
        -------
        mask : array of boolean values
            True at the indices of found targets.

        """
        if (a == b).all():
            return np.array([False]*c.shape[0])

        mask = np.array([True]*c.shape[0])
        
        dotproduct = (c[:, 0] - a[0]) * (b[0] - a[0]) + (c[:, 1] - a[1])*(b[1] - a[1])
        squaredlengthba = (b[0] - a[0])*(b[0] - a[0]) + (b[1] - a[1])*(b[1] - a[1])
        
        #exclude the targets whose vertical projection of the vector c-a w.r.t. the vector b-a is larger than the target radius.
        idx = np.argwhere(np.abs(numba.np.arraymath.cross2d(b-a, c-a))/np.linalg.norm(b-a) > r) 
        for i1 in idx:
            mask[i1] = False        
        
        #exclude the targets whose scalar product is negative (they are on the other side of the step direction)
        for i2 in np.argwhere(dotproduct < 0):
            mask[i2] = False

        #exclude the targets that are beyond the step.
        for i3 in np.argwhere(dotproduct > squaredlengthba):
            mask[i3] = False
            
        return mask

In [ ]:
#| hide
compiling = isBetween_c_vec_numba(np.array([0.1,1]), np.array([1,3]), np.random.rand(100,2), 0.00001)

In [ ]:
%timeit isBetween_c_vec_numba(np.array([0.1,1]), np.array([1,5]), np.random.rand(100,2), 0.00001)
# Run time of new version:

8.48 µs ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Crossing targets while walking along an arc

Functions to evaluate whether the robot crosses a target while reorienting itself (action turn). During reorientation, the robot walks along a circumference arc.
- rotation_arc: creates the points of the arc (how many points are created can be chosen).
- arc_in_target_numba: evaluates whether the arc points are within a target area.

In [ ]:
# |export
@jit(nopython = NOPYTHON)
def rotation_arc(theta : float, # Angle in rad (from origin, counterclockwise) describing the current orientation of the robot.
                 dtheta : float, # Angle difference in rad (with appropriate sign!! Left turn: dtheta>0, Right turn: dtheta<0). How much the robot rotates when it decides to turn.
                 num_points_arc:int=50, # Number of points that form the arc that is going to be evaluated.
                 R:float=1, # Rotation radius.
                 x:float=0, # Horizontal component of the robot's current position.
                 y:float=0 # Vertical component of the robot's current position.
                ):
    """Creates the points of the circumference arc covered by the robot while it reorients itself (action "turn")."""
    
    phi = np.pi - theta
    
    # Get origin of rotation
    origin_x = x -  R * np.cos(phi)
    origin_y = y +  R * np.sin(phi)
    
    # Generate points for the arc in rad: from start point to final point = start point + length of arc
    angles = np.linspace(-phi, -phi + dtheta, num_points_arc)
    x_arc = origin_x + R * np.cos(angles) #x positions of all the arc points
    y_arc = origin_y + R * np.sin(angles) #y positions of all the arc points
    
    return x_arc, y_arc

In [ ]:
# |export
@jit(nopython = NOPYTHON)
def arc_in_target_numba(target_coord : np.array, # Array (number targets, 2) with the coordinates of all targets.
                        target_radius : float, # Radius of each target.
                        x_arc: np.array, # Horizontal components of arc points.
                        y_arc: np.array, # Vertical components of arc points.
                       )->np.array: # 1D array of size (number targets) with 1 on the indices of the found targets.
    """
    Checks which targets have been found by the agent when it reorients itself.
    """

    # Reshape the arc arrays to make them broadcastable
    x_arc_broadcasted = x_arc[:, np.newaxis]  # Shape (num_points_arc, 1)
    y_arc_broadcasted = y_arc[:, np.newaxis]  # Shape (num_points_arc, 1)

    # Compute the squared distances from all arc points to all target centers
    distances_squared = (x_arc_broadcasted - target_coord[:, 0])**2 + (y_arc_broadcasted - target_coord[:, 1])**2

    # Check if these distances are within the target radius squared
    mask_in_target = distances_squared <= target_radius**2

    # Summing over the arc points axis (axis=0), find which targets are found
    mask_found_targets = np.sign(np.sum(mask_in_target, axis=0))
    
    return mask_found_targets.astype('bool')

In [ ]:
targ = np.random.rand(100, 2)* 100
target_r = 1
a_x, a_y = rotation_arc(np.pi/6, np.pi/3, num_points_arc=50, R=1, x=0, y=0)
compile_first = arc_in_target_numba(targ, target_r, a_x, a_y)

In [ ]:
%timeit arc_in_target_numba(targ, target_r, a_x, a_y)

8.96 µs ± 14 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Random sampling from array with probs

In [ ]:
# |export
@jit(nopython = NOPYTHON)
def rand_choice_nb(arr : np.array, # 1D numpy array of values to sample from.
                   prob : np.array # 1D numpy array of probabilities for the given samples.
                  ): # Random sample from the given array with a given probability.    
    return arr[np.searchsorted(np.cumsum(prob), np.random.random(), side="right")]

# RL agent (Projective Simulation)

We use Projective Simulation (PS) as a framework for Reinforcement Learning (RL).

## Forager 
This agent is an efficient version of rl_opts.rl_framework.Forager with:

- `numba` implementation
-  H and G matrix efficient updates (contribution by Michele Caraglio).

In [ ]:
#| export
from rl_opts.rl_framework.numba.agents import Forager

In [ ]:
f = Forager(3,np.array([100], dtype=np.int64))

In [ ]:
f.act(2)

In [ ]:
f.agent_state

0

# Environments

## Single Robot

Environment where a simulated robot (RL agent) searches for targets. The robot cannot see the targets. It gets a signal from the environment in the form of a positive reward whenever its position is within a target area.

- States: the agent perceives the counter of steps that it has been walking straight, i.e. it decided to continue in the same direction.

- Actions: "continue" , "turn left" or "turn right".
    - For "continue": walks straight in the same direction (no reorientation) for a distance $v$ sampled from a normal distribution. $N(\alpha, \sigma_\alpha)$
    - For "turning" actions: normal distribution centered on some angle to the left or to the right. $N(\beta, \sigma_\beta)$
    - When it turns, it just reorients (NO EXTRA DISPLACEMENT). The reorientation of the target detector occurs with respect to the robot's center, so (in contrast to our previous ideal agent) it also displaces its effective position when it reorients. Here, we consider that the effective position of the agent is that of the detector (not of the robot's center).

- Instead of kick, here we implement a time delay $\tau$. The agent needs to wait $\tau$ time steps from the moment it gets a target until it can get the same target again. 

- When the agent crosses a target, it resets the counter of steps walking straight. Its position doesn't change, it continues from where it was when it found the target. No extra reorientation.

In [ ]:
#|export
@jitclass([("target_positions", float64[:,:]) ,
           ("current_rewards", float64[:]) ,
           ("tau_allagents", float64[:]) ,
           ("depleted", float64[:,:]) ,
           ("current_directions", float64[:]) ,
           ("positions", float64[:,:]),
           ("previous_pos", float64[:,:]),
           ("mask", bool_[:])])
           
class RobotSearch():
    Nt : int
    L : float
    r : float
    tau : float
    agent_radius:float
    avg_vel:float
    std_vel:float
    avg_turn_angle:float
    std_turn_angle:float
    num_agents : int
    target_positions : np.ndarray
    current_rewards : np.array
    tau_allagents : np.array
    depleted : np.array
    current_directions : np.array
    positions : np.array
    previous_pos : np.array
    mask : np.array
    
    
    def __init__(self,
                 Nt:int, # Number of targets.
                 L:float, # Size of the (squared) world.
                 r:float, # Radius with center the target position. It defines the area in which agent detects the target.
                 tau:float, # Time it takes for the targets to replenish.
                 agent_radius:float, # Distance from robot's center to detector.
                 avg_vel:float, # Mean displacement of the agent per time step (v). Normal distr.: N(v,std_v).
                 std_vel:float, # Std for the normal distr. of the agent's velocity (std_v).
                 avg_turn_angle:float, # Mean turning angle (in rad) when agent decides to turn (Theta). Normal distr.: N(Theta,std_theta).
                 std_turn_angle:float, # Std for the normal distr. of the agent's turning velocity (std_theta).
                 num_agents:int=1 # Number of agents that forage at the same time.
                 ):
        
        """        
        Class defining the foraging environment with replenishable targets modeled with a time delay instead of a kick. 
        It includes the methods to simulate the robot dynamics.
        
        """
        
        
        self.Nt = Nt
        self.L = L
        self.r = r
        self.agent_radius = agent_radius
        self.avg_vel = avg_vel
        self.std_vel = std_vel
        self.avg_turn_angle = avg_turn_angle
        self.std_turn_angle = std_turn_angle
        self.num_agents = num_agents
        
        # same replenishing time for all agents
        self.tau_allagents = tau * np.ones(self.num_agents)
        

        self.init_env()
        
    def init_env(self):
        """
        Environment initialization.
        """
        self.target_positions = np.random.rand(self.Nt, 2)*self.L
        
        #store who is/was rewarded
        self.current_rewards = np.zeros(self.num_agents)
        
        #number of steps since last target encounter, for each agent.
        self.depleted = np.zeros((self.Nt, self.num_agents))
        
        #set positions and directions of the agents
        self.current_directions = np.random.rand(self.num_agents)*2*np.pi
        self.positions = np.random.rand(self.num_agents, 2)*self.L
        self.previous_pos = self.positions.copy()       

        

    def update_pos(self, action, agent_index = 0):        
        """
        Updates information of the agent depending on its decision.

        Parameters
        ----------
        action : int
            0 (continue), 1 (left), 2 (right).
        agent_index : int, optional
            Index of the given agent. The default is 0.
            
        Returns
        ----------
        (x_arc, y_arc): (np.array, np.array)
            Positions of the turning arc. In case the agent does not turn, it returns (None, None).
        
        """
        
        # Save previous position to check if crossing happened
        self.previous_pos[agent_index] = self.positions[agent_index].copy()
        
        if action == 0: #continue
            # Update the velocity of the agent
            v = np.random.normal( max(0,self.avg_vel), max(0,self.std_vel))
            # Get the positional changes
            dx = v * np.cos(self.current_directions[agent_index])
            dy = v * np.sin(self.current_directions[agent_index])
            
            x_arc = None
            y_arc = None
            
        else:
            
            if action == 1: #left turn
                # Get angle difference
                dtheta = np.random.normal(self.avg_turn_angle, self.std_turn_angle)

            elif action == 2: #right turn
                # Get angle difference
                dtheta = - np.random.normal(self.avg_turn_angle, self.std_turn_angle)

            # Get new orientation of the agent
            new_theta = self.current_directions[agent_index] + dtheta
            # Get the positional changes
            dx = self.agent_radius * (np.cos(self.current_directions[agent_index]) - np.cos(new_theta))
            dy = self.agent_radius * (np.sin(self.current_directions[agent_index]) - np.sin(new_theta))
            
            
            # Get arc coordinates (default: 50 points)
            x_arc, y_arc = rotation_arc(self.current_directions[agent_index], dtheta, R=self.agent_radius, x=self.positions[agent_index][0], y=self.positions[agent_index][1])
            
            # Update agent's orientation
            self.current_directions[agent_index] = new_theta
        
       
        # Update the position
        self.positions[agent_index][0] += dx
        self.positions[agent_index][1] += dy
        
        return x_arc, y_arc
        
       
    def check_encounter(self, action, x_arc, y_arc, agent_index = 0):
        """
        Checks whether the agent found a target, and updates the information accordingly.

        Parameters
        ----------
        action : int
            Action index. Continue (0), turn left (1) or turn right (2).
        x_arc : np.array or None
            Horizontal components of the turning arc coordinates. None if agent walked straight.
        y_arc : np.array or None
            Vertical components of the turning arc coordinates. None if agent walked straight.
        agent_index : int, optional

        Returns
        -------
        True if the agent found a target.

        """
        
        # Check if there was a target encounter
        if action == 0:
            encounters = isBetween_c_vec_numba(self.previous_pos[agent_index], self.positions[agent_index], self.target_positions, self.r)
        else:
            encounters = arc_in_target_numba(self.target_positions, self.r, x_arc, y_arc)
            
        #In case there were encounters:
        if np.sum(encounters) > 0: 
            # If there is more than 1 encounter, pick the closest to the agent.
            if sum(encounters) == 1:
                first_encounter = np.argwhere(encounters == True).flatten()
                
            else:
                # compute the distance from the previous position to each target            
                distance_previous_pos = np.sqrt((self.previous_pos[agent_index][0]- self.target_positions[:, 0])**2 + (self.previous_pos[agent_index][1] - self.target_positions[:, 1])**2)            
                # checking which encountered point is closer to previous position
                min_distance_masked = np.argmin(distance_previous_pos[encounters])
                first_encounter = np.argwhere(encounters == True)[min_distance_masked].flatten()
           
            # If the agent is NOT waiting for the recently found target to replenish:
            if not self.depleted[first_encounter, agent_index]:
                # We add the information that this agent got a reward
                self.current_rewards[agent_index] = 1
                # The replenishing time for this target and this agent starts.
                self.depleted[first_encounter, agent_index] += 1
                
                # The env issues a positive reward.
                return 1
            else:
                self.current_rewards[agent_index] = 0
                return 0
                    
        else:
            self.current_rewards[agent_index] = 0
            return 0   
    
    def update_replenishing_times(self, agent_index = 0):
        """
        Updates the replenishing times of the targets for a given agent.
        """
        self.depleted[np.argwhere(self.depleted[:, agent_index] != 0).flatten(), agent_index] += 1
        self.depleted[:, agent_index] = self.depleted[:, agent_index] % (self.tau_allagents[agent_index] + 1)
        
    def check_bc(self, agent_index = 0):
        """
        Updates position coordinates of a given agent to fulfill fixed boundary conditions.

        """
        self.positions[agent_index][0]  = max( 0, min( self.positions[agent_index][0] , self.L ) )
        self.positions[agent_index][1]  = max( 0, min( self.positions[agent_index][1] , self.L ) )

# Learning

## Training loop

In [ ]:
#| export                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
@jit(nopython = NOPYTHON)
def train_loop_robot(episodes : int, # Number of episodes to train
                   time_ep : int, # Length of episode
                   agent : object, # Agent class
                   env : object, # Environment class
                   time_steps_save_h = None #Optional: int, every time_steps_save_h, the h_matrix is stored. Default: time_ep (at the end of episode)
                  )-> tuple: # Rewards and h-matrix of the trained agent      
    '''    
    Given an agent and environment, performs a loop train for the `RobotSearch` environment.
    '''
    
    if time_steps_save_h == None:
        time_steps_save_h = time_ep
    #Initialize storage of h matrices (it overwrites every episode)
    number_stored_h_matrices = int(time_ep / time_steps_save_h)
    save_h_matrices = np.zeros((number_stored_h_matrices, agent.h_matrix.shape[-2],agent.h_matrix.shape[-1]))
    
    #Initialize storage of rewards
    save_rewards = np.zeros((episodes, time_ep))
    
    for ep in range(episodes):
        #initialize environment and agent's counter and g matrix
        env.init_env()
        agent.agent_state = 0
        agent.reset_g()

        for t in range(time_ep):
            
            
            #step to set counter to its min value n=1
            if t == 0:
                #do one step with random direction (no learning in this step)
                action = rand_choice_nb(arr = np.arange(1, agent.num_actions), prob = np.array([1/len(np.arange(1,agent.num_actions))]*len(np.arange(1,agent.num_actions))))
                _,_ = env.update_pos(action)
                #check boundary conditions
                env.check_bc()
                
            else:
                #get perception
                state = agent.get_state()
                #decide
                action = agent.deliberate(state)
                #act (update counter)
                agent.act(action)
                
                #update positions and get coordinates for turning arc (in case agent turned, otherwise these are None)
                x_arc, y_arc = env.update_pos(action)
                
                #update replenishing times
                env.update_replenishing_times()
                
                #check if target was found 
                reward = env.check_encounter(action, x_arc, y_arc)
                
                #check boundary conditions
                env.check_bc()
                
                #learn
                if reward == 1:
                    agent._learn_post_reward(reward)
                    
                    #reset counter and g matrix
                    agent.agent_state = 0
                    agent.reset_g()
                    
            #If H matrix has not been updated for max_no_H_update steps, it updates.
            if agent.N_upd_H == agent.max_no_H_update-1:
                agent._learn_post_reward(reward)
            
            #Saving time step
            if (t+1) % time_steps_save_h == 0:
                
                #Update of the H matrix, in case it was not updated in the last time step
                if agent.N_upd_H != 0:
                    agent._learn_post_reward(0)
                    
                #Save updated h matrix
                save_h_matrices[int((t+1) / time_steps_save_h -1)] = agent.h_matrix
             
            #Saving
            save_rewards[ep, t] = reward
            
        #Update of the H matrix at the end of the episode in case it was not updated in the last time steps
        if agent.N_upd_H != 0:
            agent._learn_post_reward(0)
        
    return (save_rewards, save_h_matrices)
    

In [ ]:
time_ep = 100

env = RobotSearch(Nt=100, 
                 L=100, 
                 r=1, 
                 tau=3, 
                 agent_radius=2, # Size of the agent (radius, from the center to the side).
                 avg_vel=1, # Mean displacement of the agent per time step (v). Normal distr.: N(v,std_v).
                 std_vel=0.3, # Std for the normal distr. of the agent's velocity (std_v).
                 avg_turn_angle=np.pi/6, # Mean turning angle (in rad) when agent decides to turn (Theta). Normal distr.: N(Theta,std_theta).
                 std_turn_angle=np.pi/18) # Std for the normal distr. of the agent's turning velocity (std_theta).

agent = Forager(num_actions = 3, # From here are props of the agent (see Forager for details)
                gamma_damping = 0.2,
                size_state_space = np.array([time_ep], dtype=np.int64),
                initial_prob_distr = (np.array([0.988, 0.012/2, 0.012/2])*np.ones((3, time_ep)).transpose()).transpose())


In [ ]:
rewards, h_matrices = train_loop_robot(episodes = 1, time_ep = time_ep, agent = agent, env = env, time_steps_save_h = 50)

In [ ]:
np.shape(h_matrices)

(2, 3, 100)

## Parallel training

In [ ]:
#| export
@jit(nopython = NOPYTHON, parallel = True)
def run_robot_training_parallel(episodes, # Number of episodes
                                time_ep, # Length of episode
                                N_agents, # Number of agents               
                                time_steps_save_h = None, # Every how many steps is the h matrix stored
                                Nt = 100, # From here are props of the environment (see TargetEnv for details) 
                                L = 100, 
                                r = 0.5, 
                                tau = 3,
                                agent_radius = 2,
                                avg_vel=1,
                                std_vel=0.3,
                                avg_turn_angle=np.pi/6,
                                std_turn_angle=np.pi/18,
                                num_actions = 3, # From here are props of the agent (see Forager for details)
                                size_state_space = np.array([100], dtype=np.int64), 
                                gamma_damping = 0.00001,
                                eta_glow_damping = 0.1,
                                initial_prob_distr = np.array([[],[],[]]),
                                policy_type = 'standard', 
                                beta_softmax = 3,  
                                fixed_policy = np.array([[],[]]),
                                max_no_H_update = int(1e4),
                                g_update = 's'
                               ):
    #mean reward per episode
    save_rewards = np.zeros((N_agents, episodes))
    
    #reward per time step (for single but long episodes)
    #save_rewards = np.zeros((N_agents, time_ep))
    
    #storage of h matrices
    if time_steps_save_h == None:
        time_steps_save_h = time_ep
    
    number_stored_h_matrices = int(time_ep / time_steps_save_h)
    save_h_matrix = np.zeros((N_agents, number_stored_h_matrices, num_actions, size_state_space[0]))  
     
    
    for n_agent in prange(N_agents):
        
        env = RobotSearch(Nt, L, r, tau, agent_radius, avg_vel, std_vel, avg_turn_angle, std_turn_angle,1)
        
        agent = Forager(num_actions,size_state_space,gamma_damping,
                        eta_glow_damping,policy_type,beta_softmax,
                        initial_prob_distr,fixed_policy,max_no_H_update,g_update)
                          
        rews, mat = train_loop_robot(episodes, time_ep, agent, env, time_steps_save_h)
             
        #mean reward per episode
        for t in range(episodes):
            save_rewards[n_agent, t] = np.mean(rews[t])
            
        #reward per time step (for single but long episodes)
        #save_rewards[n_agent,:] = rews[0,:]
        
        save_h_matrix[n_agent] = mat
        
    return save_rewards, save_h_matrix

In [ ]:
rewards, h_matrix = run_robot_training_parallel(episodes = 2, time_ep = 10, N_agents = 3, time_steps_save_h = 5);

In [ ]:
np.shape(h_matrix)

(3, 2, 3, 100)

# Walk from policy

These replicate what we were doing in `rl_opts.learn_and_bench.walk_from_policy` and help get efficiencies for fixed policies.

In [ ]:
#| export   
@jit(nopython = NOPYTHON)
def single_robot_trajectory(time_ep : int, # Length of each run / episode
                            policy : np.array, # Policy of the walker (rows: states. columns:prob of doing each action)
                            env : object # Environment where the walker moves
                           )-> tuple: # number of visited targets, agent and target positions

    '''
    Walk of a single agent in env of type RobotSearch given a policy. Performance is evaluated as the number of targets found in a fixed time time_ep.
    '''
    
    number_visited_targets = 0
    agent_positions = np.zeros((time_ep, 2))
    rew_per_timestep = np.zeros(time_ep)
    
    #initialize environment and agent's counter and g matrix
    env.init_env()
    agent_state = 0

    for t in range(time_ep):
        
        agent_positions[t,:] = env.positions[0] #position of agent with index 0
        
        #step to set counter to its min value n=1
        if t == 0:
            #do one step with random direction (no learning in this step)
            action = rand_choice_nb(arr = np.arange(1, len(policy[0])), prob = np.array([1/len(np.arange(1,len(policy[0])))]*len(np.arange(1,len(policy[0])))))
            _,_ = env.update_pos(action)
            #check boundary conditions
            env.check_bc()
            

        else: 
            # decide       
            action = rand_choice_nb(arr = np.arange(len(policy[0])), prob = policy[agent_state])
            #update positions
            x_arc, y_arc = env.update_pos(action)
            #update replenishing times
            env.update_replenishing_times()
            
            #check if target was found 
            reward = env.check_encounter(action, x_arc, y_arc)
            
            #check boundary conditions
            env.check_bc()
            
            # update agent_state
            if action == 0: #continue
                agent_state += 1
            else: #turn
                agent_state = 0
                
            if reward == 1:
                agent_state = 0

            number_visited_targets += reward
            rew_per_timestep[t] += reward
                
    return (number_visited_targets, agent_positions, env.target_positions, rew_per_timestep) 

In [ ]:
#| export
@jit(nopython = NOPYTHON, parallel = True)
def parallel_robots_trajectories(time_ep : int, # Length of each run / episode
                                 N_agents : int, # Number of agents to consider
                                 Nt = 100, # Number of targets in the environment
                                 L = 100, # Size of the environment
                                 r = 0.5, # Radius of the targets
                                 tau = 3, # Time delay until target is available again
                                 agent_radius = 2,
                                 avg_vel=1,
                                 std_vel=0.3,
                                 avg_turn_angle=np.pi/6,
                                 std_turn_angle=np.pi/18,
                                 policies = np.array([[1.0,0.0,0.0,0.0], [0.0,1.0,0.0,0.0]]) # Policy of the agents
                                )-> tuple: # number of visited targets, agent and target positions
    """
    Runs in parallel single_robot_trajectory. Due to numba props, we need to give all parameters as inputs (see source).
    """
    
    agent_positions = np.zeros((N_agents, time_ep, 2))
    number_visited_targets = np.zeros(N_agents)
    target_positions = np.zeros((N_agents, Nt, 2))
    reward_per_timestep = np.zeros((N_agents, time_ep))
    
    for n_agent in prange(N_agents):
        
        env = RobotSearch(Nt, L, r, tau, agent_radius, avg_vel, std_vel, avg_turn_angle, std_turn_angle,1)
        
        num_tar, pos, tar_pos, rew_t = single_robot_trajectory(time_ep, policies[n_agent], env) 
    
        agent_positions[n_agent] = pos
        number_visited_targets[n_agent] = num_tar
        target_positions[n_agent] = tar_pos
        reward_per_timestep[n_agent] = rew_t
        
    return (number_visited_targets, agent_positions, target_positions, reward_per_timestep)

#| hide
# nbdev

In [ ]:
#| hide
from nbdev import nbdev_export ; nbdev_export()